<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/17-Using_LLMs_to_rank_chunks_as_the_Judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables


In [ ]:
!pip install -q huggingface_hub llama-index==0.10.49 llama-index-vector-stores-chroma==0.1.9 llama-index-llms-gemini==0.1.11 google-generativeai==0.5.4 openai==1.35.3 chromadb==0.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
os.environ["GOOGLE_API_KEY"] = "<YOUR_API_KEY>"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
# os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')

# Load a Model


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(temperature=1, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Load Indexes


Downloading vector store from Huggingface hub

In [ ]:
from huggingface_hub import hf_hub_download
vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip",repo_type="dataset",local_dir="/content")

vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

In [ ]:
!unzip /content/vectorstore.zip

Archive:  /content/vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [ ]:
# Load the vector store from the local storage.
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db2 = chromadb.PersistentClient(path="/content/ai_tutor_knowledge")
chroma_collection = db2.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding

# Create the index based on the vector store.
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=10)

res = query_engine.query("Explain how Advance RAG works?")

In [ ]:
res.response

'Advance RAG refers to enhanced approaches within Retrieval Augmented Generation (RAG) systems, specifically focusing on methodologies like Self-Reflective RAG (Self-RAG), Corrective RAG (CRAG), and SPECULATIVE RAG.\n\n1. **Self-Reflective RAG (Self-RAG)**: This method involves instruction-tuning a large language model (LLM) to generate self-reflection tags. These tags guide the model in dynamically retrieving documents and assessing their relevance before generating responses.\n\n2. **Corrective RAG (CRAG)**: This approach employs an external evaluator to assess and refine the quality of retrieved documents prior to the generation process. It focuses on ensuring that the contextual information used during generation is of high quality.\n\n3. **SPECULATIVE RAG**: A proposed approach that further enhances RAG by employing a smaller RAG drafter model to generate multiple answer drafts based on subsets of retrieved documents. These drafts are then verified by a larger generalist LLM. This

# RankGPT


In [ ]:
from llama_index.core.postprocessor.rankGPT_rerank import RankGPTRerank

rankGPT = RankGPTRerank(top_n=3,llm=Settings.llm, verbose=True)

In [ ]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(similarity_top_k=10, node_postprocessors=[rankGPT])

res = query_engine.query("Explain how Retrieval Augmented Generation (RAG) works?")

After Reranking, new rank list for nodes: [1, 0, 3, 9, 6, 8, 4, 2, 7, 5]

In [ ]:
res.response

'Retrieval-Augmented Generation (RAG) works by integrating two main components: the Retrieval mechanism and the Generation mechanism. \n\n1. **Retrieval Mechanism**: This component is responsible for extracting relevant information from external knowledge sources, such as a dense vector index (e.g., Wikipedia). It involves:\n   - **Indexing**: Organizing documents for efficient retrieval using methods like inverted indexes or dense vector encoding.\n   - **Searching**: Utilizing the created indexes to fetch relevant documents based on user queries, often enhanced with rerankers to refine the order of returned documents.\n\n2. **Generation Mechanism**: After retrieving the relevant documents, this component generates responses based on the retrieved content and user queries. It employs a sequence-to-sequence (seq2seq) model to produce coherent and contextually appropriate outputs. This involves:\n   - Using prompting techniques like Chain of Thought and Tree of Thought to improve the ge

In [ ]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text.strip())
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 1c324686-fad7-4a41-bfd3-d44f9612ca91
Title	 Evaluation of Retrieval-Augmented Generation: A Survey:Research Paper Information
Text	 Authors: Hao Yu, Aoran Gan, Kai Zhang, Shiwei Tong, Qi Liu, and Zhaofeng Liu.Numerous studies of Retrieval-Augmented Generation (RAG) systems have emerged from various perspectives since the advent of Large Language Models (LLMs). The RAG system comprises two primary components: Retrieval and Generation. The retrieval component aims to extract relevant information from various external knowledge sources. It involves two main phases, indexing and searching. Indexing organizes documents to facilitate efficient retrieval, using either inverted indexes for sparse retrieval or dense vector encoding for dense retrieval. The searching component utilizes these indexes to fetch relevant documents based on the user's query, often incorporating the optional rerankers to refine the ranking of the retrieved documents. The generation component utilizes the retr

# Custom Postprocessor


## The `Judger` Function


The following function will query GPT-4o to retrieve the top three nodes that has highest similarity to the asked question.


In [ ]:
from pydantic import BaseModel
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate


def judger(nodes, query):

    # The model's output template
    class OrderedNodes(BaseModel):
        """A node with the id and assigned score."""

        node_id: list
        score: list

    # Prepare the nodes and wrap them in <NODE></NODE> identifier, as well as the query
    the_nodes = ""
    for idx, item in enumerate(nodes):
        the_nodes += f"<NODE{idx+1}>\nNode ID: {item.node_id}\nText: {item.text}\n</NODE{idx+1}>\n"

    query = "<QUERY>\n{}\n</QUERY>".format(query)

    # Define the prompt template
    prompt_tmpl = PromptTemplate(
        """
    You receive a qurey along with a list of nodes' text and their ids. Your task is to assign score
    to each node based on its contextually closeness to the given query. The final output is each
    node id along with its proximity score.
    Here is the list of nodes:
    {nodes_list}

    And the following is the query:
    {user_query}

    Score each of the nodes based on their text and their relevancy to the provided query.
    The score must be a decimal number between 0 an 1 so we can rank them."""
    )

    # Define the an instance of GPT-4 and send the request
    llm = OpenAI(model="gpt-4o-mini")
    ordered_nodes = llm.structured_predict(
        OrderedNodes, prompt_tmpl, nodes_list=the_nodes, user_query=query
    )

    return ordered_nodes

## Define Postprocessor


The following class will use the `judger` function to rank the nodes, and filter them based on the ranks.


In [ ]:
from typing import List, Optional
from llama_index.core import QueryBundle
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.schema import NodeWithScore


class OpenaiAsJudgePostprocessor(BaseNodePostprocessor):
    def _postprocess_nodes(
        self, nodes: List[NodeWithScore], query_bundle: Optional[QueryBundle]
    ) -> List[NodeWithScore]:

        r = judger(nodes, query_bundle)

        node_ids = r.node_id
        scores = r.score

        sorted_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        top_three_nodes = [sorted_scores[i][0] for i in range(3)]

        selected_nodes_id = [node_ids[item] for item in top_three_nodes]

        final_nodes = []
        for item in nodes:
            if item.node_id in selected_nodes_id:
                final_nodes.append(item)

        return final_nodes

In [ ]:
judge = OpenaiAsJudgePostprocessor()

## Query Engine with Postprocessor


In [ ]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(similarity_top_k=5, node_postprocessors=[judge])

res = query_engine.query("Explain how Parameter Efficient Fine tuning (PEFT) works?")

In [ ]:
res.response

'Parameter-Efficient Fine-Tuning (PEFT) focuses on adapting large pretrained models for various applications by fine-tuning only a small subset of additional model parameters. This approach significantly reduces computational and storage requirements compared to traditional full fine-tuning methods. By doing so, it allows for high-performance adaptation of models while making the process more accessible for users with consumer hardware.\n\nPEFT integrates seamlessly with popular libraries like Transformers, Diffusers, and Accelerate, optimizing both the training and inference of large models. This integration enhances user experience and efficiency in handling large language models, making advanced machine learning techniques more manageable. Users can also find resources, including quick start guides and practical how-tos, to help implement PEFT methods across diverse tasks.'

In [ ]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 ffd813ca-d987-47fa-988d-2cddaa749553
Title	 PEFT
Text	 # PEFT🤗 PEFT (Parameter-Efficient Fine-Tuning) is a library for efficiently adapting large pretrained models to various downstream applications without fine-tuning all of a model's parameters because it is prohibitively costly. PEFT methods only fine-tune a small number of (extra) model parameters - significantly decreasing computational and storage costs - while yielding performance comparable to a fully fine-tuned model. This makes it more accessible to train and store large language models (LLMs) on consumer hardware.PEFT is integrated with the Transformers, Diffusers, and Accelerate libraries to provide a faster and easier way to load, train, and use large models for inference.<div class="mt-10">  <div class="w-full flex flex-col space-y-4 md:space-y-0 md:grid md:grid-cols-2 md:gap-y-4 md:gap-x-5">    <a class="!no-underline border dark:border-gray-700 p-5 rounded-lg shadow hover:shadow-lg" href="quicktour"      ><div 